In [1]:
import os
import glob
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import estimation, simulation_room

In [17]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)
# Abrir el GSpreadsheet
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=1166684204#gid=1166684204")

In [18]:
data_input = 'input_sim' #nombre de la hoja de datos de entrada

sheet = spreadsheet.worksheet(data_input) # Abrir la hoja de cálculo con los datos de entrada
data = sheet.get_all_records()

#Ahora puedo manejar todos los datos con el DataFrame de pandas sin modificar la hoja de cálculo
df_simulations = pd.DataFrame(data)
df_simulations.head() # Verifico lo importado

,out_dir,simulation_name,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,mic_z,mic_directivity,src_dist,src_z,src_ang_start,src_ang_end,src_ang_step
0,,sweep_0_180_5_short_rev,sine_sweep_24bit.wav,20,40,10,0.8,90,4,0.1,1.2,omni,5,1.2,0,180,5
1,,sweep_0_180_5_medium_rev,sine_sweep_24bit.wav,20,40,10,1.8,90,4,0.1,1.2,omni,5,1.2,0,180,5
2,,sweep_0_180_5_long_rev,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,0,180,5


In [19]:
df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla
df_simulations.head() # Verificar el resultado

,out_dir,simulation_name,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,mic_z,mic_directivity,src_dist,src_z,src_ang_start,src_ang_end,src_ang_step,room_dim
0,,sweep_0_180_5_short_rev,sine_sweep_24bit.wav,20,40,10,0.8,90,4,0.1,1.2,omni,5,1.2,0,180,5,"(20, 40, 10)"
1,,sweep_0_180_5_medium_rev,sine_sweep_24bit.wav,20,40,10,1.8,90,4,0.1,1.2,omni,5,1.2,0,180,5,"(20, 40, 10)"
2,,sweep_0_180_5_long_rev,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,0,180,5,"(20, 40, 10)"


## Simulación de audios captados por micrófonos 

In [ ]:
for idx, sim in df_simulations.iterrows(): # Itero cada simulación
    for angle in range(sim['src_ang_start'], sim['src_ang_end'] + 1, sim['src_ang_step']):
        src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
            wav_path=f'audios/anechoic/{sim["audio"]}',
            out_dir=f'audios/output/{sim['simulation_name']}/{angle}',
            audio_name=f'{angle}_{sim["simulation_name"]}',
            fs=48000,
            room_dim=sim["room_dim"],
            rt60=sim["rt60"],
            snr_db=sim["snr_db"],
            n_mics=sim["n_mics"],
            mic_d=sim["mic_d"],
            mic_z=sim["mic_z"],
            mic_directivity=sim["mic_directivity"],
            src_dist=sim["src_dist"],
            src_az_deg=angle,
            src_z=sim["src_z"],
            save_audio=True
        )

<class 'int'>


## Calcular DOA desde archivos WAV

In [20]:
method = 'classic'  # Método de estimación a utilizar

for idx, sim in df_simulations.iterrows(): # Itero cada simulación
    # Cargar los archivos de audio generados
    simulations = os.listdir(f'audios/output/{sim["simulation_name"]}')
    # Creación del DataFrame para almacenar los resultados
    df_results = pd.DataFrame(np.repeat(df_simulations.iloc[idx:idx+1, 2:14].values, len(simulations), axis=0), columns=df_simulations.columns[2:14])

    row = 0  
    for angle in simulations:
        # Obtener las señales de cada posición simulada
        mic_signals = glob.glob(f'audios/output/{sim["simulation_name"]}/{angle}/*.wav')
        avg_angle, avg_tdoa, angles_per_mic_ref, tdoas_per_mic_ref = estimation.estimate_doa(
        signals=mic_signals,
        fs=48000,
        d=sim["mic_d"],
        method=method
        )
        
        df_results.at[row, 'method'] = method
        df_results.at[row, 'angle'] = int(angle)
        df_results.at[row, 'avg_angle'] = avg_angle
        df_results.at[row, 'angle_error'] = np.abs(avg_angle - int(angle))
        df_results.at[row, 'desv_std_angle'] = np.std([angle for angles in angles_per_mic_ref for angle in angles])
        
        row += 1
        
        
    # Ordenar valores de df
    sheet_name = f'{sim["simulation_name"]}_{method}'
    # df_results['angle'] = pd.to_numeric(df_results['angle'], errors='coerce')
    df_results = df_results.sort_values(by=['angle']).reset_index(drop=True)

    # Crear csv en una carpeta llamada csv_results
    if not os.path.exists('csv_results'):
        os.makedirs('csv_results')
        
    df_results.to_csv(f'csv_results/{sheet_name}.csv')
    # Subir a GSpreadsheet creando una nueva hoja
    
    try:
        spreadsheet.add_worksheet(title=f'{sheet_name}', rows="100", cols="20")
        results_sheet = spreadsheet.worksheet(f'{sheet_name}')
        results_sheet.update([df_results.columns.values.tolist()] + df_results.values.tolist())
    except Exception as e:
        print(f"Error al subir los resultados a Google Sheets: {e}, ver si no existe la hoja ya en el GSpreadsheet")
    
    #     # Guardar resultados en cada fila individual
        
    #     # df_results.head()
    #     row += 1
        # df_results.at[idx, 'angle_error'] = np.abs(avg_angle - int(angle))
        # df_results.at[idx, 'angle_error_pct'] = (df_results.at[idx, 'angle_error'] / angle) * 100 if sim["src_az_deg"] != 0 else 0
        # df_results.at[idx, 'std_angle'] = np.std([angle for angles in angles_per_mic_ref for angle in angles])

        # for i in range(len(angles_per_mic_ref)):
            # for j in range(len(angles_per_mic_ref[i])):
                # df_results.at[idx, f'angle_{i+1}{j+2}'] = angles_per_mic_ref[i][j]               


In [21]:
df_results.head(40)

,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,mic_z,mic_directivity,src_dist,src_z,method,angle,avg_angle,angle_error,desv_std_angle
0,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,0.0,83.195968,83.195968,1.410924e+00
1,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,5.0,13.277601,8.277601,1.086349e+01
2,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,10.0,18.899750,8.899750,3.822978e+00
3,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,15.0,19.483978,4.483978,2.938177e+00
4,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,20.0,21.726984,1.726984,6.646519e-15
5,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,25.0,29.138851,4.138851,2.438538e+00
6,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,30.0,30.962969,0.962969,9.287023e-15
7,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,35.0,36.974147,1.974147,1.575869e+00
8,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,40.0,43.105114,3.105114,1.713677e+00
9,sine_sweep_24bit.wav,20,40,10,3.0,90,4,0.1,1.2,omni,5,1.2,classic,45.0,48.279090,3.279090,2.503977e+00
